# Masakhane - Reverse Machine Translation for African Languages (Using JoeyNMT)

> ## NB
>### - The purpose of this Notebook is to build models that translate African languages(target language) *into* English(source language). This will allow us to in future be able to make translations from one African language to the other. If you'd like to translate *from* English, please use [this](https://github.com/masakhane-io/masakhane-mt/blob/master/starter_notebook.ipynb) starter notebook instead.

>### - We call this reverse training because normally we build models that make translations from the source language(English) to the target language. But in this case we are doing the reverse; building models that make translations from the target language to the source(English)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "sw" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# # This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$tgt-$src-$tag"
# os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (target_language, source_language, tag)

In [4]:
# !echo $gdrive_path

/content/drive/My Drive/masakhane/en-sw-baseline


In [3]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 80 kB 5.6 MB/s  eta 0:00:01


In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-sw.xml.gz not found. The following files are available for downloading:

        ./JW300_latest_xml_en.zip already exists
        ./JW300_latest_xml_sw.zip already exists
  10 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/en-sw.xml.gz

  10 MB Total size
./JW300_latest_xml_en-sw.xml.gz ... 100% of 10 MB
gzip: JW300_latest_xml_en-sw.xml already exists; do you wish to overwrite (y or n)? ^C


In [7]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-05-10 11:25:43--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.1’

test.en-any.en.1    100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2021-05-10 11:25:43 (7.36 MB/s) - ‘test.en-any.en.1’ saved [277791/277791]

--2021-05-10 11:25:43--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-sw.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [8]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 6478/979526 lines since contained in test set.


,source_sentence,target_sentence
0,“ Look !,“ Tazama !
1,I Am Making All Things New ”,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
2,The above is a promise from God that has been ...,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo ...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [10]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 50 kB 5.3 MB/s  eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=178072 sha256=79dd8af974a3293c3d4f3e292e1e6bc3631cff89ddd823545b39199678748f61
  Stored in directory: /home/freshiasackey_gmail_com/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [23]:
# start_time = time.time()
# ### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filter out "almost overlapping samples"
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [12]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
Besides , being single gives one the opportunity to do so many great things !
A Logical Conclusion
I still feel the pain of the loss , but thanking Jehovah for having had a good marriage and for the privilege of serving him with someone who deeply loved him has improved my outlook . ”
If we do his will , he blesses and supports us in this life and offers us the certain hope of an even better life to come .
We are determined to continue providing a rich supply of enlightening and appealing information ​ — both in print and online — ​ to benefit our many readers who respect the Bible and who want to know what it really teaches .
Well , in the past decade , the number of active Witnesses increased from slightly fewer than 3,800,000 to almost 6,000,000 .
Actually , according to the Bible , there is no spiritual part of a person that survives the death of the body .
The enemy may aim a blow by attacking Bible truths that are fundamental to your faith .
When Saul returned to



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [13]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.8.0
! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'joeynmt'...
remote: Enumerating objects: 3089, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 3089 (delta 78), reused 118 (delta 73), pack-reused 2951
Receiving objects: 100% (3089/3089), 8.07 MiB | 20.07 MiB/s, done.
Resolving deltas: 100% (2107/2107), done.
Processing /home/freshiasackey_gmail_com/joeynmt
  Created wheel for joeynmt: filename=joeynmt-1.3-py3-none-any.whl size=84842 sha256=ef07bebe91da73788bfd625d9ea9f6c2d4e41abd48a82cee881ca4aae778845b
  Stored in directory: /tmp/pip-ephem-wheel-cache-i4o_on4y/wheels/c6/e5/f6/9ca9cce28ddbc86321064c68c12df09503f033db723fa4548d
Successfully built joeynmt
  Attempting uninstall: joeynmt
    Found existing installation: joeynmt 1.3
    Uninstalling joeynmt-1.3:


      Successfully uninstalled joeynmt-1.3
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 9.8 kB/s eta 0:00:012     |██████████████████▏             | 432.3 MB 7.8 MB/s eta 0:00:43     |███████████████████▊            | 469.5 MB 2.3 MB/s eta 0:02:09
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [14]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE Swahili Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.sw     test.en-any.en.1  train.bpe.sw
dev.bpe.en	dev.sw	     test.en	     test.sw	       train.en
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.en      train.sw
cp: target '' is not a directory
cp: target '' is not a directory
cp: target '' is not a directory
cp: cannot create regular file '': No such file or directory
ls: cannot access '': No such file or directory
BPE Swahili Sentences
N@@ g@@ ao kubwa ya imani ( T@@ azama f@@ ungu la 12 hadi 14 )
K@@ of@@ ia ya chu@@ ma ya wo@@ ko@@ vu ( T@@ azama f@@ ungu la 15 hadi 18 )
N@@ im@@ et@@ ambua kwamba watu hu@@ it@@ ikia vizuri wanapo@@ ona uki@@ zungum@@ zia habari za Biblia kwa sha@@ uku na unapo@@ fanya yote una@@ yo@@ weza kuwasaidia . ”
Up@@ anga wa roho ( T@@ azama f@@ ungu la 19 na 20 )
Kwa msaada wa Yehova tunaweza kus@@ im@@ ama im@@ ara na kum@@ p@@ inga !
Combined BPE Vocab
с@@
ḥ
т@@
и
х@@
м
і@@
д@@
і
➊


In [15]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.sw     test.en-any.en.1  train.bpe.sw
dev.bpe.en	dev.sw	     test.en	     test.sw	       train.en
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.en      train.sw


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [22]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 35                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer"
    overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
"""
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-05-10 12:05:13,638 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-10 12:05:13,639 - INFO - joeynmt.data - Loading training data...
2021-05-10 12:05:33,860 - INFO - joeynmt.data - Building vocabulary...
2021-05-10 12:05:34,175 - INFO - joeynmt.data - Loading dev data...
2021-05-10 12:05:34,236 - INFO - joeynmt.data - Loading test data...
2021-05-10 12:05:34,322 - INFO - joeynmt.data - Data loaded.
2021-05-10 12:05:34,322 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-10 12:05:34,569 - INFO - joeynmt.model - Enc-dec model built.
2021-05-10 12:05:34,577 - INFO - joeynmt.training - Total params: 12219392
2021-05-10 12:05:34,580 - INFO - joeynmt.helpers - cfg.name                           : {target_language}{source_language}_reverse_transformer
2021-05-10 12:05:34,580 - INFO - joeynmt.helpers - cfg.data.src                       : sw
2021-05-10 12:05:34,580 - INFO - joeynmt.helpers - cfg.data.trg                       : en
2021-05-10 12:05:34,580

2021-05-10 12:07:14,714 - INFO - joeynmt.training - Epoch   1, Step:      100, Batch Loss:     5.824012, Tokens per Sec:     2461, Lr: 0.000300
2021-05-10 12:08:53,289 - INFO - joeynmt.training - Epoch   1, Step:      200, Batch Loss:     5.505569, Tokens per Sec:     2571, Lr: 0.000300
2021-05-10 12:10:31,850 - INFO - joeynmt.training - Epoch   1, Step:      300, Batch Loss:     5.457898, Tokens per Sec:     2620, Lr: 0.000300
2021-05-10 12:12:07,941 - INFO - joeynmt.training - Epoch   1, Step:      400, Batch Loss:     5.160206, Tokens per Sec:     2542, Lr: 0.000300
2021-05-10 12:13:44,283 - INFO - joeynmt.training - Epoch   1, Step:      500, Batch Loss:     5.064879, Tokens per Sec:     2604, Lr: 0.000300
2021-05-10 12:15:19,514 - INFO - joeynmt.training - Epoch   1, Step:      600, Batch Loss:     5.153046, Tokens per Sec:     2587, Lr: 0.000300
2021-05-10 12:16:57,236 - INFO - joeynmt.training - Epoch   1, Step:      700, Batch Loss:     4.622757, Tokens per Sec:     2603, Lr: 0

2021-05-10 12:45:54,109 - INFO - joeynmt.training - Epoch   1, Step:     2100, Batch Loss:     4.114491, Tokens per Sec:     2871, Lr: 0.000300
2021-05-10 12:47:21,622 - INFO - joeynmt.training - Epoch   1, Step:     2200, Batch Loss:     3.484627, Tokens per Sec:     2874, Lr: 0.000300
2021-05-10 12:50:18,393 - INFO - joeynmt.training - Epoch   1, Step:     2400, Batch Loss:     3.704122, Tokens per Sec:     2867, Lr: 0.000300
2021-05-10 12:51:45,707 - INFO - joeynmt.training - Epoch   1, Step:     2500, Batch Loss:     3.647293, Tokens per Sec:     2794, Lr: 0.000300
2021-05-10 12:53:14,079 - INFO - joeynmt.training - Epoch   1, Step:     2600, Batch Loss:     3.703819, Tokens per Sec:     2880, Lr: 0.000300
2021-05-10 12:54:38,979 - INFO - joeynmt.training - Epoch   1, Step:     2700, Batch Loss:     3.674768, Tokens per Sec:     2973, Lr: 0.000300
2021-05-10 12:56:04,201 - INFO - joeynmt.training - Epoch   1, Step:     2800, Batch Loss:     3.530214, Tokens per Sec:     2920, Lr: 0

2021-05-10 13:22:02,438 - INFO - joeynmt.training - Epoch   1, Step:     4200, Batch Loss:     2.996407, Tokens per Sec:     2813, Lr: 0.000300
2021-05-10 13:23:29,382 - INFO - joeynmt.training - Epoch   1, Step:     4300, Batch Loss:     3.099476, Tokens per Sec:     2794, Lr: 0.000300
2021-05-10 13:26:24,939 - INFO - joeynmt.training - Epoch   1, Step:     4500, Batch Loss:     3.238181, Tokens per Sec:     2824, Lr: 0.000300
2021-05-10 13:27:52,748 - INFO - joeynmt.training - Epoch   1, Step:     4600, Batch Loss:     3.237501, Tokens per Sec:     2835, Lr: 0.000300
2021-05-10 13:29:20,644 - INFO - joeynmt.training - Epoch   1, Step:     4700, Batch Loss:     3.250057, Tokens per Sec:     2844, Lr: 0.000300
2021-05-10 13:30:48,449 - INFO - joeynmt.training - Epoch   1, Step:     4800, Batch Loss:     3.238878, Tokens per Sec:     2818, Lr: 0.000300
2021-05-10 13:32:16,912 - INFO - joeynmt.training - Epoch   1, Step:     4900, Batch Loss:     2.947474, Tokens per Sec:     2826, Lr: 0

2021-05-10 13:57:27,905 - INFO - joeynmt.training - Epoch   1, Step:     6200, Batch Loss:     2.735085, Tokens per Sec:     2882, Lr: 0.000300
2021-05-10 13:58:55,655 - INFO - joeynmt.training - Epoch   1, Step:     6300, Batch Loss:     2.721830, Tokens per Sec:     2781, Lr: 0.000300
2021-05-10 14:00:25,340 - INFO - joeynmt.training - Epoch   1, Step:     6400, Batch Loss:     2.639565, Tokens per Sec:     2802, Lr: 0.000300
2021-05-10 14:01:53,724 - INFO - joeynmt.training - Epoch   1, Step:     6500, Batch Loss:     3.123513, Tokens per Sec:     2798, Lr: 0.000300
2021-05-10 14:03:22,714 - INFO - joeynmt.training - Epoch   1, Step:     6600, Batch Loss:     2.939059, Tokens per Sec:     2823, Lr: 0.000300
2021-05-10 14:04:51,919 - INFO - joeynmt.training - Epoch   1, Step:     6700, Batch Loss:     2.742112, Tokens per Sec:     2842, Lr: 0.000300
2021-05-10 14:06:21,332 - INFO - joeynmt.training - Epoch   1, Step:     6800, Batch Loss:     3.273561, Tokens per Sec:     2886, Lr: 0

...

2021-05-10 16:49:51,930 - INFO - joeynmt.training - Epoch   2, Step:    16300, Batch Loss:     2.335480, Tokens per Sec:     2950, Lr: 0.000300
2021-05-10 16:51:19,529 - INFO - joeynmt.training - Epoch   2, Step:    16400, Batch Loss:     2.371825, Tokens per Sec:     2872, Lr: 0.000300
2021-05-10 16:52:47,261 - INFO - joeynmt.training - Epoch   2, Step:    16500, Batch Loss:     2.606483, Tokens per Sec:     2936, Lr: 0.000300
2021-05-10 16:54:14,152 - INFO - joeynmt.training - Epoch   2, Step:    16600, Batch Loss:     2.476956, Tokens per Sec:     2838, Lr: 0.000300
2021-05-10 16:55:42,195 - INFO - joeynmt.training - Epoch   2, Step:    16700, Batch Loss:     2.273109, Tokens per Sec:     2872, Lr: 0.000300
2021-05-10 16:57:09,085 - INFO - joeynmt.training - Epoch   2, Step:    16800, Batch Loss:     2.470606, Tokens per Sec:     2869, Lr: 0.000300
2021-05-10 16:58:35,336 - INFO - joeynmt.training - Epoch   2, Step:    16900, Batch Loss:     2.239506, Tokens per Sec:     2818, Lr: 0

2021-05-10 17:24:47,373 - INFO - joeynmt.training - Epoch   2, Step:    18400, Batch Loss:     2.140292, Tokens per Sec:     2894, Lr: 0.000300
2021-05-10 17:26:15,494 - INFO - joeynmt.training - Epoch   2, Step:    18500, Batch Loss:     2.206281, Tokens per Sec:     2896, Lr: 0.000300
2021-05-10 17:27:40,942 - INFO - joeynmt.training - Epoch   2, Step:    18600, Batch Loss:     2.277930, Tokens per Sec:     2827, Lr: 0.000300
2021-05-10 17:29:06,849 - INFO - joeynmt.training - Epoch   2, Step:    18700, Batch Loss:     2.116841, Tokens per Sec:     2892, Lr: 0.000300
2021-05-10 17:30:35,663 - INFO - joeynmt.training - Epoch   2, Step:    18800, Batch Loss:     2.076362, Tokens per Sec:     2846, Lr: 0.000300
2021-05-10 17:32:03,173 - INFO - joeynmt.training - Epoch   2, Step:    18900, Batch Loss:     2.053634, Tokens per Sec:     2866, Lr: 0.000300
2021-05-10 17:33:30,952 - INFO - joeynmt.training - Epoch   2, Step:    19000, Batch Loss:     2.275691, Tokens per Sec:     2897, Lr: 0

2021-05-10 17:55:19,466 - INFO - joeynmt.training - Epoch   3, Step:    20200, Batch Loss:     2.271461, Tokens per Sec:     2915, Lr: 0.000300
2021-05-10 17:56:47,625 - INFO - joeynmt.training - Epoch   3, Step:    20300, Batch Loss:     2.046909, Tokens per Sec:     2876, Lr: 0.000300
2021-05-10 17:58:14,182 - INFO - joeynmt.training - Epoch   3, Step:    20400, Batch Loss:     2.206532, Tokens per Sec:     2864, Lr: 0.000300
2021-05-10 17:59:41,190 - INFO - joeynmt.training - Epoch   3, Step:    20500, Batch Loss:     2.080265, Tokens per Sec:     2907, Lr: 0.000300
2021-05-10 18:01:09,133 - INFO - joeynmt.training - Epoch   3, Step:    20600, Batch Loss:     2.179455, Tokens per Sec:     2891, Lr: 0.000300
2021-05-10 18:02:36,323 - INFO - joeynmt.training - Epoch   3, Step:    20700, Batch Loss:     2.198697, Tokens per Sec:     2881, Lr: 0.000300
2021-05-10 18:04:04,127 - INFO - joeynmt.training - Epoch   3, Step:    20800, Batch Loss:     2.115926, Tokens per Sec:     2858, Lr: 0

2021-05-10 18:27:42,816 - INFO - joeynmt.training - Epoch   3, Step:    22100, Batch Loss:     2.179298, Tokens per Sec:     2884, Lr: 0.000300
2021-05-10 18:29:09,642 - INFO - joeynmt.training - Epoch   3, Step:    22200, Batch Loss:     1.911644, Tokens per Sec:     2919, Lr: 0.000300
2021-05-10 18:30:36,544 - INFO - joeynmt.training - Epoch   3, Step:    22300, Batch Loss:     2.098760, Tokens per Sec:     2874, Lr: 0.000300
2021-05-10 18:32:02,872 - INFO - joeynmt.training - Epoch   3, Step:    22400, Batch Loss:     2.001364, Tokens per Sec:     2844, Lr: 0.000300
2021-05-10 18:33:29,896 - INFO - joeynmt.training - Epoch   3, Step:    22500, Batch Loss:     1.917742, Tokens per Sec:     2880, Lr: 0.000300
2021-05-10 18:34:56,473 - INFO - joeynmt.training - Epoch   3, Step:    22600, Batch Loss:     1.971986, Tokens per Sec:     2844, Lr: 0.000300
2021-05-10 18:36:24,135 - INFO - joeynmt.training - Epoch   3, Step:    22700, Batch Loss:     2.091261, Tokens per Sec:     2924, Lr: 0

2021-05-10 19:00:40,537 - INFO - joeynmt.training - Epoch   3, Step:    24100, Batch Loss:     1.761699, Tokens per Sec:     2854, Lr: 0.000300
2021-05-10 19:02:06,905 - INFO - joeynmt.training - Epoch   3, Step:    24200, Batch Loss:     1.903977, Tokens per Sec:     2923, Lr: 0.000300
2021-05-10 19:03:32,696 - INFO - joeynmt.training - Epoch   3, Step:    24300, Batch Loss:     1.905390, Tokens per Sec:     2903, Lr: 0.000300
2021-05-10 19:04:59,193 - INFO - joeynmt.training - Epoch   3, Step:    24400, Batch Loss:     2.093480, Tokens per Sec:     2926, Lr: 0.000300
2021-05-10 19:06:25,690 - INFO - joeynmt.training - Epoch   3, Step:    24500, Batch Loss:     2.135311, Tokens per Sec:     2954, Lr: 0.000300
2021-05-10 19:07:51,921 - INFO - joeynmt.training - Epoch   3, Step:    24600, Batch Loss:     2.122672, Tokens per Sec:     2913, Lr: 0.000300
2021-05-10 19:09:18,775 - INFO - joeynmt.training - Epoch   3, Step:    24700, Batch Loss:     2.238747, Tokens per Sec:     2899, Lr: 0

In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "$gdrive_path/models/${src}${tgt}_reverse_transformer/"

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/models/${tgt}${src}_reverse_transformer/validations.txt"

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${tgt}${src}_reverse_transformer/config.yaml"